**get model coordinates**

In [2]:
import plotly.express as px
import pandas as pd
import numpy as np

coor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/distan_lat42.58lon-8.8046p3R4Km.csv")
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

**get station and sample max gust hour before**

In [3]:
station = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/station_data/station.csv", usecols = ["time","gust_o"],parse_dates=["time"]).set_index("time")
print("raw data\n",station.head(24))

#show duplicated
df_duplicate = station[station.index.duplicated()]
print("duplicated")
print(df_duplicate)

#delete duplicated and resample
station = station[~station.index.duplicated(keep='first')]
station = station.resample("H",label="right",closed="right").max()

print("Max gust hour before")
print(station.head(4))
print("Variable statistics")
print(station.describe())

raw data
                         gust_o
time                          
2018-01-01 00:10:00   6.666672
2018-01-01 00:20:00   6.138894
2018-01-01 00:30:00   7.472228
2018-01-01 00:40:00   6.833339
2018-01-01 00:50:00   7.333339
2018-01-01 01:00:00   8.527785
2018-01-01 01:10:00   9.138896
2018-01-01 01:20:00   8.000006
2018-01-01 01:30:00   8.777785
2018-01-01 01:40:00   9.222230
2018-01-01 01:50:00   8.361118
2018-01-01 02:00:00   8.583340
2018-01-01 02:10:00   9.583341
2018-01-01 02:20:00   9.222230
2018-01-01 02:30:00   8.638896
2018-01-01 02:40:00   9.694452
2018-01-01 02:50:00   8.638896
2018-01-01 03:00:00   8.472229
2018-01-01 03:10:00   8.055562
2018-01-01 03:20:00   8.250007
2018-01-01 03:30:00  11.416676
2018-01-01 03:40:00  12.583343
2018-01-01 03:50:00   9.055563
2018-01-01 04:00:00  10.916675
duplicated
                        gust_o
time                          
2018-03-25 03:00:00   2.527780
2018-03-25 03:10:00   6.166672
2018-03-25 03:20:00   6.222227
2018-03-25 03:30:0

**get meteorologic model d0**

In [4]:
#ria forecast d0
met_model = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/lat42.58lon-8.8046p3R4KmD1.csv",parse_dates=["time"]).drop(columns=["Unnamed: 0"]).set_index("time")
met_model.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43776 entries, 2018-01-02 00:00:00 to 2023-01-02 23:00:00
Data columns (total 75 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dir0         43776 non-null  float64
 1   snow_prec0   43776 non-null  float64
 2   snowlevel0   43776 non-null  float64
 3   mod0         43776 non-null  float64
 4   wind_gust0   43776 non-null  float64
 5   mslp0        43776 non-null  float64
 6   temp0        43776 non-null  float64
 7   rh0          43776 non-null  float64
 8   visibility0  43776 non-null  float64
 9   lhflx0       43776 non-null  float64
 10  lwflx0       43776 non-null  float64
 11  conv_prec0   43776 non-null  float64
 12  prec0        43776 non-null  float64
 13  swflx0       43776 non-null  float64
 14  shflx0       43776 non-null  float64
 15  cape0        43776 non-null  float64
 16  cin0         43776 non-null  float64
 17  cfh0         43776 non-null  float64
 18  cfl0       

**Concatenate and dropnan**

In [5]:
df_all = pd.concat([station,met_model],axis=1).dropna()
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43084 entries, 2018-01-02 00:00:00 to 2022-12-31 00:00:00
Data columns (total 76 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gust_o       43084 non-null  float64
 1   dir0         43084 non-null  float64
 2   snow_prec0   43084 non-null  float64
 3   snowlevel0   43084 non-null  float64
 4   mod0         43084 non-null  float64
 5   wind_gust0   43084 non-null  float64
 6   mslp0        43084 non-null  float64
 7   temp0        43084 non-null  float64
 8   rh0          43084 non-null  float64
 9   visibility0  43084 non-null  float64
 10  lhflx0       43084 non-null  float64
 11  lwflx0       43084 non-null  float64
 12  conv_prec0   43084 non-null  float64
 13  prec0        43084 non-null  float64
 14  swflx0       43084 non-null  float64
 15  shflx0       43084 non-null  float64
 16  cape0        43084 non-null  float64
 17  cin0         43084 non-null  float64
 18  cfh0       

**best correlation**

In [6]:
df_all.corr()["gust_o"].sort_values(ascending=False).head(10)

gust_o        1.000000
mod0          0.746662
wind_gust1    0.740745
mod1          0.730406
wind_gust0    0.727396
wind_gust2    0.723387
mod2          0.712500
lhflx2        0.291039
lhflx1        0.283806
lhflx0        0.251568
Name: gust_o, dtype: float64

**Mean absolute error in m/s**

In [7]:
from sklearn.metrics import mean_absolute_error
import plotly.express as px

print("mean absolute error mod0:", round(mean_absolute_error(df_all.gust_o, df_all.mod0),2))
print("mean absolute error wg1:", round(mean_absolute_error(df_all.gust_o, df_all.wind_gust1),2))
print("mean absolute error wg0:", round(mean_absolute_error(df_all.gust_o, df_all.wind_gust0),2))
print("mean absolute error wg2:", round(mean_absolute_error(df_all.gust_o, df_all.wind_gust2),2))
print("mean absolute error mod1:", round(mean_absolute_error(df_all.gust_o, df_all.mod1),2))

diff =  df_all["wind_gust1"]-df_all["gust_o"]
print("\nDifferences wind gust1 and gust observed")
print(diff.describe())
print("\nDifferences")
fig = px.line(diff)
fig.show()


mean absolute error mod0: 3.62
mean absolute error wg1: 2.49
mean absolute error wg0: 2.57
mean absolute error wg2: 2.58
mean absolute error mod1: 3.1

Differences wind gust1 and gust observed
count    43084.000000
mean        -0.121659
std          3.245258
min        -19.152608
25%         -2.184829
50%         -0.354112
75%          1.719551
max         19.429616
dtype: float64

Differences


**LGBMRegressor**

In [8]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.gust_o
X = df_all[["wind_gust0","mod0","wind_gust1","mod1","wind_gust2","mod2","prec0"]]

MAE = []
for t in range (0,20):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()  
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE":MAE,});
df_r.describe()


machine learning number: 0
            y_test       y_pred         diff
count  4309.000000  4309.000000  4309.000000
mean      7.716117     7.603495    -0.112622
std       4.310345     3.467942     2.646354
min       0.000000     1.783445   -13.340569
25%       4.111114     4.653755    -1.722715
50%       7.305561     7.087537    -0.054073
75%      10.722231     9.972089     1.565011
max      26.027799    19.759875    10.348147
machine learning number: 1
            y_test       y_pred         diff
count  4309.000000  4309.000000  4309.000000
mean      7.721442     7.728829     0.007387
std       4.411952     3.546333     2.640888
min       0.000000     1.999310   -16.982406
25%       4.055559     4.713721    -1.600497
50%       7.166672     7.161313     0.078768
75%      10.972231    10.191801     1.675160
max      29.361135    21.300043    12.774582
machine learning number: 2
            y_test       y_pred         diff
count  4309.000000  4309.000000  4309.000000
mean      7.643775 

Mean absolute value all machine learning models


,MAE
count,20.000000
mean,2.015000
std,0.026258
min,1.950000
25%,2.007500
50%,2.020000
75%,2.030000
max,2.060000


**Shapiro test normality MAE samples tested**

In [9]:
from scipy.stats import shapiro

sta, p = shapiro(MAE)
print("\np value:",p)
print("significance level alpha=0.01")

alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable MAE in the samples")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable MAE no normal distribution in samples")  


p value: 0.23483498394489288
significance level alpha=0.01
Accept null hypothesis: normal distribution variable MAE in the samples


**T test (1sample) compare machine learning mean absolute error in a sample (n=20) versus meteorological model mean absolute error at point 1  (wind_gust1 = 2.49). The alternative hypothesis is: the sample mean absolute error of  the sample is less than the given population mean absolute error (popmean). Significance level (alpha) = 0.01**

In [12]:
from scipy import stats

alpha = 0.01
sta0, p_value0 = stats.ttest_1samp(MAE, popmean=2.49, alternative="less")

print("MAE in test sample:",MAE)
print("p value:",p_value0)

if p_value0 < alpha:    
    print("\nwe are rejecting null hypothesis  H0: Mean Absolute Error machine learning = Mean Absolute Error meteorological model wind_gust1")
    print("\nwe are accepting alternative hypothesis: Mean Absolute Error machine learning is less than Mean Absolute Error meteorological model wind_gust1")
else:
    print("\nwe are accepting null hypothesis:  H0:  Mean Absolute Error machine learning = Mean Absolute Error meteorological model wind_gust1")

MAE in test sample: [2.03, 2.02, 2.02, 2.05, 2.03, 1.99, 2.02, 2.06, 2.01, 1.99, 2.02, 1.97, 2.0, 2.03, 2.01, 2.05, 1.95, 2.01, 2.02, 2.02]
p value: 6.942960349212826e-26

we are rejecting null hypothesis  H0: Mean Absolute Error machine learning = Mean Absolute Error meteorological model wind_gust1

we are accepting alternative hypothesis: Mean Absolute Error machine learning is less than Mean Absolute Error meteorological model wind_gust1


**Add time variables**

In [13]:

df_all["hour"] = df_all.index.hour
df_all["month"] = df_all.index.month
df_all["dayofyear"] = df_all.index.dayofyear
df_all["weekofyear"] = df_all.index.isocalendar().week.astype(int)

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.gust_o
X = df_all[["wind_gust0","mod0","wind_gust1","mod1","wind_gust2","mod2","hour",
            "month","dayofyear","weekofyear"]]

MAE_t = []
for t in range (0,20):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE_t.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()  
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE_t":MAE_t,});
df_r.describe()



machine learning number: 0
            y_test       y_pred         diff
count  4309.000000  4309.000000  4309.000000
mean      7.716117     7.633153    -0.082964
std       4.310345     3.501017     2.467652
min       0.000000     1.700400   -13.816446
25%       4.111114     4.677369    -1.543064
50%       7.305561     7.112146    -0.001579
75%      10.722231    10.075679     1.473346
max      26.027799    20.809484    10.274415
machine learning number: 1
            y_test       y_pred         diff
count  4309.000000  4309.000000  4309.000000
mean      7.721442     7.740598     0.019156
std       4.411952     3.553846     2.463899
min       0.000000     1.979371   -16.340337
25%       4.055559     4.716354    -1.419972
50%       7.166672     7.196116     0.140629
75%      10.972231    10.244744     1.537838
max      29.361135    20.476249    13.920642
machine learning number: 2
            y_test       y_pred         diff
count  4309.000000  4309.000000  4309.000000
mean      7.643775 

Mean absolute value all machine learning models


,MAE_t
count,20.000000
mean,1.898000
std,0.023753
min,1.850000
25%,1.890000
50%,1.900000
75%,1.910000
max,1.940000


**T test two related samples. This is a test for the null hypothesis that two related samples have identical mean absolute error. Alternative hypothesis the  mean absolute error from the sample with time variables is less than the mean absolute error of from the sample with time variables. Significance level (alpha) = 0.01**

In [14]:
from scipy import stats

alpha = 0.01
sta, p_value = stats.ttest_rel(MAE, MAE_t, alternative="greater")

print("\nMAE without time variables:", np.round(MAE,3))
print("\nMAE with time variables:", np.round(MAE_t,3))
print("\nalpha: ",alpha)
print("\np_value point: ",p_value)

if p_value < alpha:    
    print("\nwe are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables")
    print("\nwe are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs")
else:
    print("\nwe are accepting null hypothesis:  H0: MAE machine learning without time variables = MAE machine learning with time variables")


MAE without time variables: [2.03 2.02 2.02 2.05 2.03 1.99 2.02 2.06 2.01 1.99 2.02 1.97 2.   2.03
 2.01 2.05 1.95 2.01 2.02 2.02]

MAE with time variables: [1.89 1.87 1.9  1.94 1.91 1.89 1.9  1.94 1.9  1.88 1.91 1.85 1.9  1.91
 1.89 1.92 1.85 1.89 1.91 1.91]

alpha:  0.01

p_value point:  2.077131368255048e-20

we are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables

we are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs


**Save algorithm**

In [15]:
import pickle
from sklearn.pipeline import Pipeline


estimators = [('ml_model', ml_model)]
pipe = Pipeline(estimators)
print(pipe)
al_file={"x_var":X.columns,"coor":coor,"pipe":pipe}
pickle.dump(al_file, open("/content/drive/MyDrive/Colab Notebooks/Coron/algorithms/gust_coron_d1.al", 'wb'))

Pipeline(steps=[('ml_model', LGBMRegressor(n_estimators=200))])
